# 15 Plot Top of Soil Moisture #

In [1]:
%load_ext autoreload
%autoreload 2
    
import os
import sys
import pandas as pd
import xarray as xr
import numpy as np
from dask.distributed import Client
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm, TwoSlopeNorm, CenteredNorm
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
sys.path.append('/home/563/sc1326/repos/cdrmip_extremes')
from cdrmip_extremes.configs import data_dir, models, expts
from cdrmip_extremes import utils, load_data

In [2]:
client = Client(memory_limit=None,threads_per_worker=1,n_workers=28)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/40207/status,
Dashboard: /proxy/40207/status,Workers: 28
Total threads: 28,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37871,Workers: 0
Dashboard: /proxy/40207/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:44999,Total threads: 1
Dashboard: /proxy/33847/status,Memory: 0 B
Nanny: tcp://127.0.0.1:36735,


## Load Data ##

In [3]:
data = load_data.load_mrsos()

In [4]:
data

{'ACCESS-ESM1-5': {'1pctCO2': <xarray.DataArray 'mrsos' (time: 1800, lat: 145, lon: 192)> Size: 200MB
  [50112000 values with dtype=float32]
  Coordinates:
    * time     (time) object 14kB 0101-01-16 12:00:00 ... 0250-12-16 12:00:00
    * lat      (lat) float64 1kB -90.0 -88.75 -87.5 -86.25 ... 87.5 88.75 90.0
    * lon      (lon) float64 2kB 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
      depth    float64 8B ...
  Attributes:
      long_name:      Moisture in Upper Portion of Soil Column
      standard_name:  mass_content_of_water_in_soil_layer
      cell_measures:  area: areacella
      cell_methods:   area: mean where land time: mean
      comment:        The mass of water in all phases in the upper 10cm of the ...
      units:          kg m-2
      history:        2019-11-15T08:55:55Z altered by CMOR: Treated scalar dime...,
  '1pctCO2-cdr': <xarray.DataArray 'mrsos' (time: 9120, lat: 145, lon: 192)> Size: 1GB
  [253900800 values with dtype=float32]
  Coordinates:
    * tim

In [5]:
gwl_years = load_data.load_gwl_years()

## Concatenate and separate into GWL periods ##

In [ ]:
concat = {}
for model in models:
    ds_up = data[model]['1pctCO2']
    ds_down = data[model]['1pctCO2-cdr']
    concat[model] = utils.concat_branches(
        ds_up,
        ds_down
    )

In [ ]:
gwl_periods = {}
for model, ds in concat.items():
    gwl_periods[model] = utils.extract_gwl_period(
        ds,
        gwl_years[model],
        21,
        time_dim='time'
    ) 

In [ ]:
slices_15 = {}
gwl=1.5
for model, ds in gwl_periods.items():
    slices_15[model] = utils.compare_gwl_means(ds.sel(gwl=gwl))
differences_15 = {model: 100*(slices_15[model]['difference']/slices_15[model]['ramp_down']) for model in models}

## Plot differences at 1.5ºC ##

In [ ]:
def plot_gwl_diff(to_plot,levels):

    cmap = plt.cm.RdYlBu_r.copy()
    # cmap.set_extremes(under='midnightblue',over='maroon')
          


    fig, axes = plt.subplots(2,4, subplot_kw={"projection":ccrs.Robinson()},
                         figsize=(17,6),
                         sharey=True,)

    cbar_kwargs = {"fraction": 0.02, "aspect": 20,
                    "label":"Soil Moisture Change",'orientation':'vertical',
                    'location':'right','pad':0.01,
                  'anchor':(0.5,-4), 'extend':'both'
                  }
 
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
    
    for (model, da), ax in zip(to_plot.items(), axes.flat):
        im = ax.imshow(to_plot[model],
                       transform=ccrs.PlateCarree(),
                       cmap=cmap,
                       origin='lower',
                       extent=(0,360,-90,90),
                       norm=norm
                      )
        ax.set_title(model,fontsize=16)
        ax.coastlines()

    cbar_ax = fig.add_axes([1.01,0.05,0.01,0.8])
    cbar = fig.colorbar(im,
                        cax=cbar_ax,
                        **cbar_kwargs
                       )
    cbar.ax.tick_params(labelsize=14) 
    cbar.set_label("Relative Soil Moisture Difference (%)", fontsize=16)
    fig.suptitle(f"Top soil moisture difference ramp-down - ramp-up at 1.5°C GWL",
                 fontsize=25,
                 y=1.0
                )
    fig.tight_layout()
    plt.show()

In [ ]:
plot_gwl_diff(
    differences_15,
    np.linspace(-10,10,21)
)